In [4]:
#import dependencies
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
import pandas as pd
import os
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float,ForeignKey 

import pymysql
pymysql.install_as_MySQLdb()

In [4]:
data_df=pd.read_csv("Cleaned_Data.csv",low_memory=False)

data_df=data_df.rename(columns={"Price Maximum": "PriceMaximum", "Price Minimum": "PriceMinimum","Start Date":"StartDate"})
#data_df=data_df.drop(['Unnamed: 10'], axis=1)
data_df.head()

,Genre,Latitude,Longitude,Name,PriceMaximum,PriceMinimum,StartDate,Subgenre,Venue,VenueID
0,Rock,41.506036,-81.699570,Taylor Swift reputation Stadium Tour,495.5,45.5,7/17/18,Pop,"""FirstEnergy Stadium, Home of the Cleveland Br...",KovZpZAFdEJA
1,Metal,41.542609,-87.771175,Ozzy Osbourne: No More Tours 2,250.0,29.5,9/21/18,Heavy Metal,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
2,Rock,41.542609,-87.771175,Lynyrd Skynyrd: Last of the Street Survivors F...,199.5,29.5,8/3/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
3,Rock,41.542609,-87.771175,Imagine Dragons: EVOLVE TOUR,146.0,35.0,7/13/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA
4,Pop,41.542609,-87.771175,The Adventures of Kesha and Macklemore,137.0,30.5,7/14/18,Pop,"""Hollywood Casino Amphitheatre - Chicago, IL""",KovZpZAEktFA


In [5]:
#declare the base
Base=declarative_base()

#declare classes and attributes
class Events(Base):
    
    __tablename__="Events"
    id=Column(Integer,primary_key=True)
    genre=Column(String(255))
    Latitude = Column(Float)
    Longitude = Column(Float)
    Name=Column(String(255))
    PriceMaximum = Column(Float)
    PriceMinimum = Column(Float)
    
    StartDate=Column(String(255))
    Subgenre = Column(String(255))
    Venue = Column(String(255))
    VenueID = Column(String(255))
    
   

In [6]:
engine = create_engine(os.path.join("sqlite:///EventsDB.db"))
conn = engine.connect()
Base.metadata.create_all(conn)
session = Session(bind=engine)

In [7]:
data_df.to_sql('Events',conn,if_exists = 'append',index = False)
session.commit()

conn.close()

In [5]:
engine = create_engine(os.path.join("sqlite:///EventsDB.db"),echo=False)


conn = engine.connect()

In [6]:
Base = automap_base()
# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)
# Print all of the classes mapped to the Base
Base.classes.keys()

['Events']

In [7]:
events_db = Base.classes.Events

session = Session(engine)

In [9]:
first_row = session.query(events_db).first()
first_row.__dict__

{'Latitude': 41.50603554,
 'Longitude': -81.69956958,
 'Name': 'Taylor Swift reputation Stadium Tour',
 'PriceMaximum': 495.5,
 'PriceMinimum': 45.5,
 'StartDate': '7/17/18',
 'Subgenre': 'Pop',
 'Venue': '"FirstEnergy Stadium, Home of the Cleveland Browns"',
 'VenueID': 'KovZpZAFdEJA',
 '_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1b01f3e57f0>,
 'genre': 'Rock',
 'id': 1}

In [52]:
result_sample = session.query(events_db.Latitude,events_db.Longitude,events_db.Name,events_db.PriceMaximum,\
                       events_db.PriceMinimum,events_db.StartDate,events_db.Subgenre,events_db.Venue).all()

In [53]:
result_sample[0]

(41.50603554,
 -81.69956958,
 'Taylor Swift reputation Stadium Tour',
 495.5,
 45.5,
 '7/17/18',
 'Pop',
 '"FirstEnergy Stadium, Home of the Cleveland Browns"')

In [56]:
result_list=[]
result_dict={}
names = ["Latitude", "Longitude", "Name","PriceMaximum","PriceMinimum","StartDate","Subgenre","Venue"]
for each in result_sample:
    result_dict={}
    for i in range(len(names)):
        result_dict[names[i]]=each[i]
        print(each[1])
    result_list.append(result_dict)

-81.69956958
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-87.7711753
-90.46995600000001
-90.469956000

-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-115.175233
-92.7182728
-92.7182728
-92.7182728
-92.7182728
-92.7182728
-92.7182728
-92.7182728
-92.7182728
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-81.69956958
-75.164767
-75.164767
-75.164767
-75.164767
-75.164767
-75.164767
-75.164767
-75.164767
-83.04661800000001
-83.04661800000001
-83.04661800000001
-83.04661800000001
-83.04661800000001
-83.04661800000001
-83.04661800000001
-83.04661800000001
-86.8055152
-86.8055

-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999
-115.16889909999999


-90.20122536
-90.20122536
-90.20122536
-90.20122536
-90.20122536
-90.20122536
-90.20122536
-90.20122536
-90.20122536
-90.20122536
-81.504616
-81.504616
-81.504616
-81.504616
-81.504616
-81.504616
-81.504616
-81.504616
-118.34186799999999
-118.34186799999999
-118.34186799999999
-118.34186799999999
-118.34186799999999
-118.34186799999999
-118.34186799999999
-118.34186799999999
-117.0058537
-117.0058537
-117.0058537
-117.0058537
-117.0058537
-117.0058537
-117.0058537
-117.0058537
-73.753957
-73.753957
-73.753957
-73.753957
-73.753957
-73.753957
-73.753957
-73.753957
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-121.500231
-83.38022990000002
-83.38022990000002
-83.38022990000002
-83.38022990000002
-83.38022990000002
-83.38022990000002
-83.38022990000002
-83.38022990000002
-86.77857
-86.77857
-86.77857
-86.77857
-86.77857
-86.77857
-86.77857
-86.77857
-68.77

-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115.169756
-115

...

-78.619991
-78.619991
-78.619991
-75.134686
-75.134686
-75.134686
-75.134686
-75.134686
-75.134686
-75.134686
-75.134686
-80.845519
-80.845519
-80.845519
-80.845519
-80.845519
-80.845519
-80.845519
-80.845519
-83.00741409999999
-83.00741409999999
-83.00741409999999
-83.00741409999999
-83.00741409999999
-83.00741409999999
-83.00741409999999
-83.00741409999999
-80.0111627
-80.0111627
-80.0111627
-80.0111627
-80.0111627
-80.0111627
-80.0111627
-80.0111627
-95.36381697
-95.36381697
-95.36381697
-95.36381697
-95.36381697
-95.36381697
-95.36381697
-95.36381697
-77.061339
-77.061339
-77.061339
-77.061339
-77.061339
-77.061339
-77.061339
-77.061339
-76.30546488
-76.30546488
-76.30546488
-76.30546488
-76.30546488
-76.30546488
-76.30546488
-76.30546488
-90.297912
-90.297912
-90.297912
-90.297912
-90.297912
-90.297912
-90.297912
-90.297912
-85.75788997
-85.75788997
-85.75788997
-85.75788997
-85.75788997
-85.75788997
-85.75788997
-85.75788997
-73.98501292
-73.98501292
-73.98501292
-73.98501292
-73

-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-122.2860047
-90.064749
-90.064749
-90.064749
-90.064749
-90.064749
-90.064749
-90.064749
-90.064749
-117.871605
-117.871605
-117.871605
-117.871605
-117.871605
-117.871605
-117.871605
-117.871605
-105.208969
-105.208969
-105.208969
-105.208969
-105.208969
-105.208969
-105.208969
-105.208969
-117.871605
-117.871605
-117.871605
-117.871605
-117.871605
-117.871605
-117.871605
-117.871605
-116.20713400000001
-116.20713400000001
-116.20713400000001
-116.20713400000001
-116.20713400000001
-116.20713400000001
-116.20713400000001
-116.20713400000001
-116.166
-116.166
-116.166
-116.166
-116.166
-116.166
-116.166
-116.166
-112.033096
-112.033096
-112.033096
-112.033096
-112.033096
-112.033096
-112.033096
-112.033096
-105.228104
-105.228104
-105.228104
-105.228104
-105.228104
-105.228104
-105.228104
-105.228104
-96.871399
-96.871399
-96.87139

-78.8762239
-78.8762239
-78.8762239
-78.8762239
-115.17831299999999
-115.17831299999999
-115.17831299999999
-115.17831299999999
-115.17831299999999
-115.17831299999999
-115.17831299999999
-115.17831299999999
-117.4225336
-117.4225336
-117.4225336
-117.4225336
-117.4225336
-117.4225336
-117.4225336
-117.4225336
-118.3612821
-118.3612821
-118.3612821
-118.3612821
-118.3612821
-118.3612821
-118.3612821
-118.3612821
-117.13138300000001
-117.13138300000001
-117.13138300000001
-117.13138300000001
-117.13138300000001
-117.13138300000001
-117.13138300000001
-117.13138300000001
-90.066064
-90.066064
-90.066064
-90.066064
-90.066064
-90.066064
-90.066064
-90.066064
-81.407097
-81.407097
-81.407097
-81.407097
-81.407097
-81.407097
-81.407097
-81.407097
-104.995071
-104.995071
-104.995071
-104.995071
-104.995071
-104.995071
-104.995071
-104.995071
-112.079228
-112.079228
-112.079228
-112.079228
-112.079228
-112.079228
-112.079228
-112.079228
-90.082802
-90.082802
-90.082802
-90.082802
-90.082802
-

-108.5429
-108.5429
-108.5429
-122.2584
-122.2584
-122.2584
-122.2584
-122.2584
-122.2584
-122.2584
-122.2584
-86.790001
-86.790001
-86.790001
-86.790001
-86.790001
-86.790001
-86.790001
-86.790001
-97.00520300000001
-97.00520300000001
-97.00520300000001
-97.00520300000001
-97.00520300000001
-97.00520300000001
-97.00520300000001
-97.00520300000001
-93.286102
-93.286102
-93.286102
-93.286102
-93.286102
-93.286102
-93.286102
-93.286102
-117.2593
-117.2593
-117.2593
-117.2593
-117.2593
-117.2593
-117.2593
-117.2593
-112.094902
-112.094902
-112.094902
-112.094902
-112.094902
-112.094902
-112.094902
-112.094902
-70.307503
-70.307503
-70.307503
-70.307503
-70.307503
-70.307503
-70.307503
-70.307503
-83.059799
-83.059799
-83.059799
-83.059799
-83.059799
-83.059799
-83.059799
-83.059799
-77.019798
-77.019798
-77.019798
-77.019798
-77.019798
-77.019798
-77.019798
-77.019798
-117.12930300000001
-117.12930300000001
-117.12930300000001
-117.12930300000001
-117.12930300000001
-117.12930300000001
-1

-77.434601
-77.434601
-94.571297
-94.571297
-94.571297
-94.571297
-94.571297
-94.571297
-94.571297
-94.571297
-106.797203
-106.797203
-106.797203
-106.797203
-106.797203
-106.797203
-106.797203
-106.797203
-84.16190300000001
-84.16190300000001
-84.16190300000001
-84.16190300000001
-84.16190300000001
-84.16190300000001
-84.16190300000001
-84.16190300000001
-76.694801
-76.694801
-76.694801
-76.694801
-76.694801
-76.694801
-76.694801
-76.694801
-87.6605
-87.6605
-87.6605
-87.6605
-87.6605
-87.6605
-87.6605
-87.6605
-77.294899
-77.294899
-77.294899
-77.294899
-77.294899
-77.294899
-77.294899
-77.294899
-122.33429699999999
-122.33429699999999
-122.33429699999999
-122.33429699999999
-122.33429699999999
-122.33429699999999
-122.33429699999999
-122.33429699999999
-119.681602
-119.681602
-119.681602
-119.681602
-119.681602
-119.681602
-119.681602
-119.681602
-77.294899
-77.294899
-77.294899
-77.294899
-77.294899
-77.294899
-77.294899
-77.294899
-87.6605
-87.6605
-87.6605
-87.6605
-87.6605
-87.6

In [57]:
result_list

[{'Latitude': 41.50603554,
  'Longitude': -81.69956958,
  'Name': 'Taylor Swift reputation Stadium Tour',
  'PriceMaximum': 495.5,
  'PriceMinimum': 45.5,
  'StartDate': '7/17/18',
  'Subgenre': 'Pop',
  'Venue': '"FirstEnergy Stadium, Home of the Cleveland Browns"'},
 {'Latitude': 41.542609399999996,
  'Longitude': -87.7711753,
  'Name': 'Ozzy Osbourne: No More Tours 2',
  'PriceMaximum': 250.0,
  'PriceMinimum': 29.5,
  'StartDate': '9/21/18',
  'Subgenre': 'Heavy Metal',
  'Venue': '"Hollywood Casino Amphitheatre - Chicago, IL"'},
 {'Latitude': 41.542609399999996,
  'Longitude': -87.7711753,
  'Name': 'Lynyrd Skynyrd: Last of the Street Survivors Farewell Tour',
  'PriceMaximum': 199.5,
  'PriceMinimum': 29.5,
  'StartDate': '8/3/18',
  'Subgenre': 'Pop',
  'Venue': '"Hollywood Casino Amphitheatre - Chicago, IL"'},
 {'Latitude': 41.542609399999996,
  'Longitude': -87.7711753,
  'Name': 'Imagine Dragons: EVOLVE TOUR',
  'PriceMaximum': 146.0,
  'PriceMinimum': 35.0,
  'StartDate': '7